In [11]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.Strategy.MVO import *
from strategy_v2.Strategy.MVO.AlphaModel import *
from strategy_v2.Strategy.MVO.RiskModel import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *
from utils.ta import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
instruments = [
    'META',
    'TSLA',
    'NVDA',
    'AAPL',
    #'EWY',
    'DXJ',
    'BRK-B',
    'SPY',
    'QQQ',
    'NANC',
    #'BTC',
    #'DJT',
]

end_date = get_today()
start_date = pd.to_datetime(datetime(2024, 1, 3))
start_date = pd.to_datetime(datetime(2023, 2, 7))
max_leverage = 1
#vol_target = 0.25
vol_target = None

# Notes

- 2024-08-28: Tested expected return prediction with RandomForest, XGB, LGBM. None of them outperforms the SMA Model. LGBM is able to achieve a similar performance as SMA model and generally train faster.</br>

    | Measure                | ^SPX      | MVO - SMA1 | MVO - RandomForest1 | MVO - XGB1 | MVO - LGBM1 |
    |------------------------|-----------|------------|---------------------|------------|-------------|
    | Cumulative Return      | 1.173503  | 2.072994   | 1.662573            | 1.471717   | 2.023378    |
    | Annualized Return      | 0.076775  | 0.301537   | 0.221185            | 0.176047   | 0.296252    |
    | Annualized Volatility  | 0.180081  | 0.226417   | 0.240133            | 0.243953   | 0.243120    |
    | Annualized Sharpe Ratio| 0.240438  | 1.183923   | 0.781685            | 0.584416   | 1.080844    |
    | Maximum Drawdown       | -0.254251 | -0.271732  | -0.323658           | -0.380737  | -0.296535   |

    Models are shared the same hyperparameters: lookback (train days) = 10 days and gamma=10, hhi=0.2


## Leverage 
##### 1. capital (Stock MV / last leverage), 2. new leverage
2024-09-06: $169,336, 96.69%

2024-09-20: $172,168, 93.52%

2024-10-04: $176,235, 100%

2024-10-18: $177,831, 100%

2024-11-01: $175,558, 100%

2024-11-09: $181,511

2024-11-22: $187,224

2024-11-30: $188,332

2024-12-06: $194,828

# Log

2024-11-22: If we don't fix sum of weights to 1, the model would statisfy the HHI constraints by reducing the overeall weights only, but not diversing the portfolios. Still, fixing to 1 is not good because sometime we should de-leverage given signals are weak. So we can consider to add a cash element in the optimization

2024-11-30: Create benchmark portfolios (e.g. equally weighted of all stocks)

In [27]:
portfolio = PortfolioStandard(
    capital=194828,
    name='MVOPortfolio',        
    rebalance_iter=RebalancerIter('0 0 * * Fri', 1),        
    tc_model=TransactionCostFutu(), 
    systems_style=SystemStyle.VERTICAL,
    systems=[        
        # Long terms signals => 60days return / diversified        
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(60), risk_model=RollingMeanCovNeg(60), gamma=10, hhi=0.2, confidence=1, leverage=1)], max_leverage=max_leverage, offset=60),

        # Mid terms signals => 30days return / diversified        
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(30), risk_model=RollingMeanCovNeg(30), gamma=10, hhi=0.2, confidence=1, leverage=1)], max_leverage=max_leverage, offset=60),

        # Short terms signals => 5 and 10days returns / less diversified
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(10), risk_model=RollingMeanCovNeg(10), gamma=30, hhi=0.2, confidence=0.5, leverage=1)], max_leverage=max_leverage, offset=60),         
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(5), risk_model=RollingMeanCovNeg(5), gamma=30, hhi=0.2, confidence=0.5, leverage=1)], max_leverage=max_leverage, offset=60),         

        # Mean Revert signals => 2days RSI / concentrated
        #TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RSI(2,10), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=0.5)], max_leverage=max_leverage, offset=200),
        #TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RSI(2,95), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=-0.5)], max_leverage=max_leverage, offset=200),

        # Mean Revert signals => Double7 / concentrated
        #TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=Double7(5), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=0.5)], max_leverage=max_leverage, offset=200),                        
    ]
)

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize()
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

2024-12-07 03:04:56,663 - TradingSubSystemSingle [MVO - RollingMean(60)|1] - INFO - Generating position for strategy MVO - RollingMean(60)|1 between 2023-02-06 and 2024-11-29......
100%|██████████| 475/475 [00:02<00:00, 158.41it/s]
2024-12-07 03:04:59,723 - TradingSubSystemSingle [MVO - RollingMean(60)|1] - INFO - Volatility Target = nan% | Price Volatility = 33.8% | Last Scale Factor = 1.00
2024-12-07 03:04:59,725 - TradingSubSystemSingle [MVO - RollingMean(30)|1] - INFO - Generating position for strategy MVO - RollingMean(30)|1 between 2023-02-06 and 2024-11-29......
100%|██████████| 475/475 [00:04<00:00, 97.57it/s] 
2024-12-07 03:05:04,611 - TradingSubSystemSingle [MVO - RollingMean(30)|1] - INFO - Volatility Target = nan% | Price Volatility = 25.9% | Last Scale Factor = 1.00
2024-12-07 03:05:04,620 - TradingSubSystemSingle [MVO - RollingMean(10)|0.5] - INFO - Generating position for strategy MVO - RollingMean(10)|0.5 between 2023-02-06 and 2024-11-29......
100%|██████████| 475/475 

,Rebalanced Portfolio,Optimized Portfolio,^SPX,MVO - RollingMean(60)|1,MVO - RollingMean(30)|1,MVO - RollingMean(10)|0.5,MVO - RollingMean(5)|0.5
Measure,,,,,,,
Cumulative Return,2.529375,2.338434,1.451222,2.626685,2.310174,1.564305,1.433740
Annualized Return,0.537608,0.494732,0.213270,0.564060,0.490649,0.252834,0.204313
Annualized Volatility,0.225967,0.227804,0.125136,0.249468,0.239073,0.109546,0.105522
Annualized Sharpe Ratio,2.248360,2.042016,1.468151,2.142590,1.928686,2.038246,1.656153
Maximum Drawdown,-0.144863,-0.159181,-0.102766,-0.179179,-0.172660,-0.069425,-0.069066


In [28]:
portfolio.portfolio_breakdown()
portfolio.instrument_breakdown()

In [29]:
portfolio.get_position_for_trade().tail(20)

2024-12-07 03:05:12,671 - Standard Portfolio ({self.name}) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $194,828
2024-12-07 03:05:12,672 - Standard Portfolio ({self.name}) - INFO - Generate trade position based on target capital of $194,828


,AAPL,BRK-B,DXJ,META,NANC,NVDA,QQQ,SPY,TSLA
Date,,,,,,,,,
2024-11-01,28.0,36.0,311.0,33.0,402.0,310.0,20.0,23.0,166.0
2024-11-04,28.0,36.0,311.0,33.0,402.0,310.0,20.0,23.0,166.0
2024-11-05,28.0,36.0,311.0,33.0,402.0,310.0,20.0,23.0,166.0
2024-11-06,28.0,36.0,311.0,33.0,402.0,310.0,20.0,23.0,166.0
2024-11-07,28.0,36.0,311.0,33.0,402.0,310.0,20.0,23.0,166.0
2024-11-08,1.0,22.0,143.0,21.0,478.0,270.0,32.0,26.0,218.0
2024-11-11,1.0,22.0,143.0,21.0,478.0,270.0,32.0,26.0,218.0
2024-11-12,1.0,22.0,143.0,21.0,478.0,270.0,32.0,26.0,218.0
2024-11-13,1.0,22.0,143.0,21.0,478.0,270.0,32.0,26.0,218.0


# Execute the Portfolio

In [24]:
executor = ExecutorFutu(is_test=False)
executor.set_portfolio(portfolio)
orders = executor.execute(px_interval='5m')
orders

2024-12-07 02:58:23,026 - ExecutorFutu - INFO - market: US
2024-12-07 02:58:23,058 - ExecutorFutu - INFO - Cancel all orders first before executing.....


2024-12-07 02:58:23,086 | 80243 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=12, host=127.0.0.1, port=11111, user_id=18214795
2024-12-07 02:58:23,260 | 80243 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=12


2024-12-07 02:58:26,267 - Standard Portfolio ({self.name}) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $194,828
2024-12-07 02:58:26,274 - Standard Portfolio ({self.name}) - INFO - Generate trade position based on target capital of $194,828
2024-12-07 02:58:26,286 - ExecutorFutu - INFO - Execute Standard Portfolio ({self.name}) position based on 2024-11-29


2024-12-07 02:58:26,292 | 80243 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=13, host=127.0.0.1, port=11111, user_id=18214795


2024-12-07 02:58:26,317 - Futu - INFO - 9 Positions: US.TSLA, US.SPY, US.QQQ, US.NVDA, US.NANC, US.EWY, US.DXJ, US.BRK.B, US.AAPL


...

2024-12-07 02:58:50,137 - Futu - INFO - Placed Order: {'code': 'US.SPY', 'price': 607.95, 'qty': 12.0, 'trd_side': 'BUY', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2024-12-07 02:58:50,142 | 80243 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=21
2024-12-07 02:58:53,685 | 80243 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=22, host=127.0.0.1, port=11111, user_id=18214795


2024-12-07 02:58:54,013 - Futu - INFO - Placed Order: {'code': 'US.TSLA', 'price': 377.39, 'qty': 29.0, 'trd_side': 'SELL', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2024-12-07 02:58:54,017 | 80243 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=22


,code,stock_name,trd_side,order_type,order_status,order_id,qty,price,create_time,updated_time,...,remark,time_in_force,fill_outside_rth,aux_price,trail_type,trail_value,trail_spread,currency,portfolio,date
0,US.AAPL,苹果,BUY,NORMAL,SUBMITTING,5099517339716264065,21.0,243.51,2024-12-06 13:58:27,2024-12-06 13:58:27,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-12-06
0,US.BRK.B,伯克希尔-B,SELL,NORMAL,SUBMITTING,6748358462169087278,12.0,470.43,2024-12-06 13:58:30,2024-12-06 13:58:30,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-12-06
0,US.DXJ,日本对冲股票ETF-WisdomTree,BUY,NORMAL,SUBMITTING,4429436492223108599,18.0,110.92,2024-12-06 13:58:33,2024-12-06 13:58:33,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-12-06
0,US.META,Meta Platforms,BUY,NORMAL,SUBMITTING,4595824723851484371,20.0,625.10,2024-12-06 13:58:36,2024-12-06 13:58:36,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-12-06
0,US.NANC,UNUSUAL WHALES SUBVERSIVE DEMOCRATIC TRADING ETF,BUY,NORMAL,SUBMITTING,7851656651293477179,154.0,40.62,2024-12-06 13:58:40,2024-12-06 13:58:40,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-12-06
0,US.NVDA,英伟达,BUY,NORMAL,SUBMITTING,7561917903201969892,17.0,142.42,2024-12-06 13:58:43,2024-12-06 13:58:43,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-12-06
0,US.QQQ,纳指100ETF-Invesco QQQ Trust,BUY,NORMAL,SUBMITTING,1925659762251632210,19.0,526.21,2024-12-06 13:58:46,2024-12-06 13:58:46,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-12-06
0,US.SPY,标普500ETF-SPDR,BUY,NORMAL,SUBMITTING,720935337917169376,12.0,607.95,2024-12-06 13:58:50,2024-12-06 13:58:50,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-12-06
0,US.TSLA,特斯拉,SELL,NORMAL,SUBMITTING,8334266159898859958,29.0,377.39,2024-12-06 13:58:53,2024-12-06 13:58:53,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2024-12-06


# Calibration Portfolios